In [4]:
import os
import pandas as pd
DATA_PATH = os.getenv("DATA_PATH")

In [5]:
def list_and_compare_files(path_old, path_new):
    try:
        # lijst met alle nieuwe en oude filenames
        filenames_old = os.listdir(path_old)
        filenames_new = os.listdir(path_new)

        # gelijke filenames
        common_filenames = set(filenames_old).intersection(filenames_new)

        return filenames_old, filenames_new, common_filenames
    except Exception as e:
        return f"An error occurred: {e}"

# Define the paths for the 'old' and 'new' folders
path_old = os.path.join(DATA_PATH, "old")  # Replace with the actual path
path_new = os.path.join(DATA_PATH, "new")  # Replace with the actual path

# Get the results
filenames_old, filenames_new, common_filenames = list_and_compare_files(path_old, path_new)

In [6]:
import pandas as pd

# dicts voor kolommen in df
data = {'Old Filenames': [], 'New Filenames': []}

# gelijke filenamen toevoegen
for file in sorted(common_filenames):
    data['Old Filenames'].append(file)
    data['New Filenames'].append(file)

# Add remaining files from each list
for file in filenames_old:
    if file not in common_filenames:
        data['Old Filenames'].append(file)
        data['New Filenames'].append(None)

for file in filenames_new:
    if file not in common_filenames:
        data['Old Filenames'].append(None)
        data['New Filenames'].append(file)

# Create a DataFrame
df = pd.DataFrame(data)

df

,Old Filenames,New Filenames
0,Account activiteitscode.csv,Account activiteitscode.csv
1,Account.csv,Account.csv
2,Activiteit vereist contact.csv,Activiteit vereist contact.csv
3,Activiteitscode.csv,Activiteitscode.csv
4,Afspraak betreft contact_cleaned.csv,Afspraak betreft contact_cleaned.csv
5,Afspraak_account_gelinkt_cleaned.csv,Afspraak_account_gelinkt_cleaned.csv
6,CDI mailing.csv,CDI mailing.csv
7,CDI sent email clicks.csv,CDI sent email clicks.csv
8,CDI visits.csv,CDI visits.csv
9,Campagne.csv,Campagne.csv
